In [7]:
import csv
import requests
import time

# Step 1: Read tickers from CSV
with open('tickers.csv', 'r') as f:
    reader = csv.reader(f)
    tickers = [row[0].strip().upper() for row in reader if row]

# Step 2: Output field names
output_fields = [
    "company_ticker", "isin", "company_name", "sector", "industry",
    "industry_code", "industry_description"
]

# Step 3: Open output file and write header
with open('filtered_companies.csv', 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=output_fields)
    writer.writeheader()

    # Step 4: Query each ticker individually
    for ticker in tickers:
        url = f"https://sasb.ifrs.org/wp-json/sasb/v1/companySearch?s={ticker}"
        headers = {"User-Agent": "Mozilla/5.0"}

        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"❌ Failed for {ticker}: HTTP {response.status_code}")
                # Still write an empty row with ticker
                writer.writerow({"company_ticker": ticker})
                continue

            companies = response.json()
            if companies:
                # Write only the first matched company
                company = companies[0]
                row = {key: company.get(key, "") for key in output_fields}
                writer.writerow(row)
                print(f"✔ Saved data for {ticker}")
            else:
                # No company found — write empty row with ticker
                writer.writerow({"company_ticker": ticker})
                print(f"⚠ No data for {ticker}, wrote empty row.")

        except Exception as e:
            print(f"❗ Error processing {ticker}: {e}")
            writer.writerow({"company_ticker": ticker})

        time.sleep(0.5)  # polite delay

print("✅ Done. All data written to 'filtered_companies.csv'")


✔ Saved data for BHARTI
✔ Saved data for ZOMATO
✔ Saved data for INFOE
✔ Saved data for IDEA
✔ Saved data for TCOM
✔ Saved data for SUNTV
✔ Saved data for AFFLE
✔ Saved data for TTLS
✔ Saved data for Z
✔ Saved data for RAILTEL
✔ Saved data for ROUTE
✔ Saved data for NETM
✔ Saved data for TV18
✔ Saved data for EASEMYTR
✔ Saved data for JUST
✔ Saved data for SARE
⚠ No data for TIPSMUSI, wrote empty row.
✔ Saved data for NAZARA
✔ Saved data for DBCL
✔ Saved data for TEAM
✔ Saved data for HATH
✔ Saved data for NELI
✔ Saved data for DITV
✔ Saved data for CARTRADE
✔ Saved data for PRIF
✔ Saved data for MPS
⚠ No data for PRAVEG, wrote empty row.
✔ Saved data for DEN
✔ Saved data for YATRA
✔ Saved data for JAGP
✔ Saved data for OPIL
✔ Saved data for MTNL
✔ Saved data for SIGNPOST
✔ Saved data for GTPL
✔ Saved data for GFL
✔ Saved data for MSIL
✔ Saved data for TTAN
✔ Saved data for TTMT
✔ Saved data for BJAUT
✔ Saved data for MM
✔ Saved data for TRENT
✔ Saved data for EIM
✔ Saved data for TVSL

In [8]:
import csv
import requests
import time

input_file = 'filtered_companies.csv'
output_file = 'filtered_companies_retry.csv'

# API headers
headers = {"User-Agent": "Mozilla/5.0"}

# Fields you're using
output_fields = [
    "company_ticker", "isin", "company_name", "sector", "industry",
    "industry_code", "industry_description"
]

# Read original file
rows = []
with open(input_file, 'r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        rows.append(row)

# Update missing rows
for row in rows:
    ticker = row["company_ticker"].strip().upper()

    # If 'isin' is already present, retain data as-is
    if row["isin"].strip():
        print(f"✅ Skipped {ticker} (already filled)")
        continue

    # Retry API call
    url = f"https://sasb.ifrs.org/wp-json/sasb/v1/companySearch?s={ticker}"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            companies = response.json()
            if companies:
                company = companies[0]
                for key in output_fields:
                    row[key] = company.get(key, "")
                print(f"🔁 Retried and filled data for {ticker}")
            else:
                print(f"⚠ No company found for {ticker}")
        else:
            print(f"❌ HTTP {response.status_code} for {ticker}")
        time.sleep(0.5)
        
    except Exception as e:
        print(f"❗ Error retrying {ticker}: {e}")

# Write updated data to output file
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=output_fields)
    writer.writeheader()
    writer.writerows(rows)

print(f"✅ Retry complete. Updated file written to '{output_file}'")


✅ Skipped BHARTI (already filled)
✅ Skipped ZOMATO (already filled)
✅ Skipped INFOE (already filled)
✅ Skipped AGS (already filled)
✅ Skipped 8071 (already filled)
✅ Skipped SUNTV (already filled)
✅ Skipped AFFLE (already filled)
✅ Skipped TTLS (already filled)
✅ Skipped Z (already filled)
✅ Skipped RAILTEL (already filled)
✅ Skipped NSG (already filled)
✅ Skipped 251270 (already filled)
✅ Skipped ALAMAR (already filled)
✅ Skipped EASEMYTR (already filled)
✅ Skipped JUST (already filled)
✅ Skipped WRLS (already filled)
⚠ No company found for TIPSMUSI
✅ Skipped NAZARA (already filled)
✅ Skipped DBCL (already filled)
✅ Skipped 3662 (already filled)
✅ Skipped BRK/A (already filled)
✅ Skipped CNLI (already filled)
✅ Skipped DITV (already filled)
✅ Skipped CARTRADE (already filled)
✅ Skipped PRIF (already filled)
✅ Skipped ALEXSC (already filled)
⚠ No company found for PRAVEG
✅ Skipped 138SL (already filled)
✅ Skipped YATRA (already filled)
✅ Skipped JAGP (already filled)
✅ Skipped OPIL (al

In [10]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Input and output paths
input_file = 'filtered_companies_retry.csv'
output_file = 'company_sasb_issues.csv'

# List of all 26 issues
all_issues = [
    "GHG Emissions", "Air Quality", "Energy Management", "Water & Wastewater Management",
    "Waste & Hazardous Materials Management", "Ecological Impacts", "Human Rights & Community Relations",
    "Customer Privacy", "Data Security", "Access & Affordability", "Product Quality & Safety", "Customer Welfare",
    "Selling Practices & Product Labeling", "Labour Practices", "Employee Health & Safety",
    "Employee Engagement, Diversity & Inclusion", "Product Design & Lifecycle Management",
    "Business Model Resilience", "Supply Chain Management", "Materials Sourcing & Efficiency",
    "Physical Impacts of Climate Change", "Business Ethics", "Competitive Behaviour",
    "Management of the Legal & Regulatory Environment", "Critical Incident Risk Management",
    "Systemic Risk Management"
]

# HTTP headers
headers = {"User-Agent": "Mozilla/5.0"}

# Load input companies
df = pd.read_csv(input_file)

# Open output CSV and write header
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['Ticker'] + all_issues
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for step, (_, row) in enumerate(df.iterrows(), start=1):
        ticker = row["company_ticker"]
        isin = row["isin"]
        url = f"https://sasb.ifrs.org/standards/materiality-finder/find/?company[0]={isin}"

        print(f"[Step {step}] Fetching for {ticker}...")

        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'lxml')

            ul = soup.find("ul", class_="relevant-issues-list")
            if not ul:
                print(f"⚠ [Step {step}] No relevant issues list found for {ticker}")
                writer.writerow({"Ticker": ticker})  # Empty row with just ticker
                continue

            # Extract bold issues
            bold_issues = set(strong.get_text(strip=True) for strong in ul.find_all("strong"))

            # Build row data
            result = {"Ticker": ticker}
            for issue in all_issues:
                result[issue] = 1 if issue in bold_issues else ""

            writer.writerow(result)
            print(f"✔ [Step {step}] Written data for {ticker}")

        except Exception as e:
            print(f"❗ [Step {step}] Error for {ticker}: {e}")
            writer.writerow({"Ticker": ticker})  # Write fallback row

        time.sleep(0.5)  # Polite delay

print("✅ Done. Data saved to 'company_sasb_issues.csv'")


[Step 1] Fetching for BHARTI...
✔ [Step 1] Written data for BHARTI
[Step 2] Fetching for ZOMATO...
✔ [Step 2] Written data for ZOMATO
[Step 3] Fetching for INFOE...
✔ [Step 3] Written data for INFOE
[Step 4] Fetching for AGS...
✔ [Step 4] Written data for AGS
[Step 5] Fetching for 8071...
✔ [Step 5] Written data for 8071
[Step 6] Fetching for SUNTV...
✔ [Step 6] Written data for SUNTV
[Step 7] Fetching for AFFLE...
✔ [Step 7] Written data for AFFLE
[Step 8] Fetching for TTLS...
✔ [Step 8] Written data for TTLS
[Step 9] Fetching for Z...
✔ [Step 9] Written data for Z
[Step 10] Fetching for RAILTEL...
✔ [Step 10] Written data for RAILTEL
[Step 11] Fetching for NSG...
✔ [Step 11] Written data for NSG
[Step 12] Fetching for 251270...
✔ [Step 12] Written data for 251270
[Step 13] Fetching for ALAMAR...
✔ [Step 13] Written data for ALAMAR
[Step 14] Fetching for EASEMYTR...
✔ [Step 14] Written data for EASEMYTR
[Step 15] Fetching for JUST...
✔ [Step 15] Written data for JUST
[Step 16] Fetchin

In [12]:
! pip install openpyxl


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: C:\Users\Uday Om\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Input and output files
csv_file = "company_sasb_issues.csv"
excel_file = "company_sasb_issues_highlighted.xlsx"

# Load the CSV file
df = pd.read_csv(csv_file)

# Get list of SASB issue columns (all except 'Ticker')
sasb_columns = df.columns.tolist()[1:]

# Save initial Excel file
df.to_excel(excel_file, index=False)

# Load workbook for styling
wb = load_workbook(excel_file)
ws = wb.active

# Define yellow fill
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Iterate over DataFrame rows
for idx, row in df.iterrows():
    # Check if all SASB columns are empty
    if row[sasb_columns].isnull().all() or (row[sasb_columns] == "").all():
        excel_row = idx + 2  # Add 2 because Excel is 1-based and row 1 is header
        for col_idx in range(1, len(df.columns) + 1):
            ws.cell(row=excel_row, column=col_idx).fill = yellow_fill

# Save styled Excel file
wb.save(excel_file)
print(f"✅ Highlighted rows saved in '{excel_file}'")


✅ Highlighted rows saved in 'company_sasb_issues_highlighted.xlsx'


In [15]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Files
xlsx_file = "company_sasb_issues_highlighted.xlsx"
csv_file = "tickers.csv"
output_file = "company_sasb_issues_mismatch_highlighted.xlsx"

# Load tickers.csv (list of expected tickers)
with open(csv_file, 'r') as f:
    expected_tickers = [line.strip().upper() for line in f if line.strip()]

# Load Excel file (tickers to validate)
df_excel = pd.read_excel(xlsx_file)
actual_tickers = df_excel["Ticker"].astype(str).str.upper().tolist()

# Load workbook for styling
wb = load_workbook(xlsx_file)
ws = wb.active

# Define light red fill
red_fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")

# Compare tickers one by one
for idx, (expected, actual) in enumerate(zip(expected_tickers, actual_tickers), start=2):  # Excel row index starts at 2
    if expected != actual:
        for col in range(1, len(df_excel.columns) + 1):
            ws.cell(row=idx, column=col).fill = red_fill
        print(f"❌ Mismatch at row {idx}: expected '{expected}', found '{actual}'")
    else:
        print(f"✔ Match at row {idx}: {expected}")

# Save to new Excel file
wb.save(output_file)
print(f"\n✅ Done. Mismatched rows saved to '{output_file}'")


✔ Match at row 2: BHARTI
✔ Match at row 3: ZOMATO
✔ Match at row 4: INFOE
❌ Mismatch at row 5: expected 'IDEA', found 'AGS'
❌ Mismatch at row 6: expected 'TCOM', found '8071'
✔ Match at row 7: SUNTV
✔ Match at row 8: AFFLE
✔ Match at row 9: TTLS
✔ Match at row 10: Z
✔ Match at row 11: RAILTEL
❌ Mismatch at row 12: expected 'ROUTE', found 'NSG'
❌ Mismatch at row 13: expected 'NETM', found '251270'
❌ Mismatch at row 14: expected 'TV18', found 'ALAMAR'
✔ Match at row 15: EASEMYTR
✔ Match at row 16: JUST
❌ Mismatch at row 17: expected 'SARE', found 'WRLS'
✔ Match at row 18: TIPSMUSI
✔ Match at row 19: NAZARA
✔ Match at row 20: DBCL
❌ Mismatch at row 21: expected 'TEAM', found '3662'
❌ Mismatch at row 22: expected 'HATH', found 'BRK/A'
❌ Mismatch at row 23: expected 'NELI', found 'CNLI'
✔ Match at row 24: DITV
✔ Match at row 25: CARTRADE
✔ Match at row 26: PRIF
❌ Mismatch at row 27: expected 'MPS', found 'ALEXSC'
✔ Match at row 28: PRAVEG
❌ Mismatch at row 29: expected 'DEN', found '138SL'
